In [1]:
import numpy as np 
from uncertainties import ufloat, unumpy
import pandas as pd

In [2]:
data=pd.read_csv('Daten/links_zsm.csv',delimiter=';',decimal=".") #importiere Daten
data.columns=['Nm','Nm_error']
Nm_value = data['Nm'].to_numpy() 
Nm_error = data['Nm_error'].to_numpy()


data=pd.read_csv('Daten/rechts_zsm.csv',delimiter=';',decimal=".") #importiere Daten
data.columns=['Np','Np_error']
Np_value = data['Np'].to_numpy() 
Np_error = data['Np_error'].to_numpy()


print(np.mean(Nm_error),np.mean(Np_error))
#-> sys abweichung von N+- ist etwa 120


119.46439260720487 117.75744111082047


In [3]:
#Methode 1
#Summe der Messerwerte bilden, daraus die Asymmetrie E berechnen. Fehler der Asymmetrie aus gausscher Fehlerfortpflanzung, fehler der Summe = Wurzel der Summe, Methode 1
Np = np.sum(Np_value)
sigma_sys_N = 120
sigma_Np = np.sqrt(Np)
print("Summe N_+ = ", Np, "+- ", sigma_Np)
Nm = np.sum(Nm_value)
sigma_Nm = np.sqrt(Nm)
print("Summe N_- = ", Nm, "+- ", sigma_Nm)
E_1 = (Nm-Np)/(Nm+Np)
E_1_sigma_stat = np.sqrt((2*Nm*sigma_Np/(Nm+Np)**2)**2+ (2*Np*sigma_Nm/(Nm+Np)**2)**2)
#Annahme : Systematischer Fehler bei Summe der Zähler etwa 120*30
E_1_sigma_sys = np.sqrt((2*Nm*sigma_sys_N/(Nm+Np)**2)**2+ (2*Np*sigma_sys_N/(Nm+Np)**2)**2)
print("Asymmetrie E = ", E_1, "+- ", E_1_sigma_stat , "+- ", E_1_sigma_sys)


Summe N_+ =  416099.0 +-  645.057361790407
Summe N_- =  428262.0 +-  654.4172980598847
Asymmetrie E =  0.01440497607066172 +-  0.0010881552300383707 +-  0.00020100790279674462


In [4]:
#Methode 2
#Asymmetrie der Einzelwerte bilden, daraus mittelwert und standartabweichung = stat fehler, sys fehler wie bei methode 1

E_2 = (Nm_value - Np_value)/(Nm_value + Np_value)
E_2_sigma_stat = np.std(E_2)/np.sqrt(30)
E_2 = np.mean(E_2)

#Annahme: Systematischer Fehler bei einzelner Messung etwa 120, siehe Zelle 1
E_2_sigma_sys = np.sqrt(2*(1+E_1**2))/(Nm+Np) * 100

print(E_2, "+- ", E_2_sigma_stat ,"+- ", E_2_sigma_sys)


0.01437539361841089 +-  0.0010868184413687198 +-  0.00016750658566395385


In [5]:
#Berechnung der Polarisation, index 1 & 2 jeweils für Methode 1 und zwei
E = np.array([E_1,E_2])
f= 2/26
phi = 0.52
E_sigma_stat = np.array([E_1_sigma_stat,E_2_sigma_stat])
E_sigma_sys = np.array([E_1_sigma_sys,E_2_sigma_sys])

P = E/(2/26*0.52)

#statistischer Fehler
P_sigma_stat = E_sigma_stat/(f*phi)

#systematischer Fehler, gaussche Fehlerfortpflanzung (die erste ableitung nach E, die zweite ist ableitung nach phi/phi_0 
#dritter fehler auf f, mit fehler auf E berechtet, fehler auf phi = 0.05, fehler auf f = 0.007
P_sigma_sys = np.sqrt((1/(f*phi)*E_sigma_sys)**2 + (-E/(f*phi**2)*0.05)**2 + (E/(f**2*phi)*0.007)**2)

print("Polarisation mit Methode 1: P= ",P[0] , "+- ", P_sigma_stat[0], "+-", P_sigma_sys[0])
print("Polarisation mit Methode 2: P= ",P[1] , "+- ", P_sigma_stat[1], "+-", P_sigma_sys[1])




Polarisation mit Methode 1: P=  0.360124401766543 +-  0.027203880750959267 +- 0.04794022502381883
Polarisation mit Methode 2: P=  0.35938484046027225 +-  0.027170461034217996 +- 0.04776214866065227


In [6]:
#Berechnung der Helizität, Schätzung L = 0.8 +- 0.15
L = 0.8
L_sigma = 0.15
H = P/L
H_sigma_stat = np.sqrt((P_sigma_stat/L)**2)
H_sigma_sys = np.sqrt((1/L*P_sigma_sys)**2 + ((-P/L**2)*L_sigma)**2)

print("Helizität mit Methode 1: P= ",H[0] , "+- ", H_sigma_stat[0], "+-", P_sigma_sys[0])
print("Helizität mit Methode 2: P= ",H[1] , "+- ", H_sigma_stat[1], "+-", P_sigma_sys[1])


Helizität mit Methode 1: P=  0.4501555022081787 +-  0.03400485093869908 +- 0.04794022502381883
Helizität mit Methode 2: P=  0.4492310505753403 +-  0.033963076292772496 +- 0.04776214866065227
